In [1]:
function get_score(contract,trick_made,is_vul=false) # get the raw score of any contract
    trump=contract[2]
    trick_bid= parse(Int32,contract[1])+6
    level=parse(Int32,contract[1])
    made= trick_made >= trick_bid
    over_trick = trick_made - trick_bid
    if made
        if trump in ['H','S']
            contract_points=(trick_made-6)*30
            is_game=level >=4
        elseif trump in ['D','C']
            contract_points=(trick_made-6)*20
            is_game=level >=5
        else
            contract_points=(40+(trick_made-6-1)*30)
            is_game=level >=3
        end
        game_points=0
        slam_bonus=0
        part_score_bonus=0
        if is_game
            game_points=300+is_vul*200
        else
            part_score_bonus=50
        end
        if level==6
            slam_bonus=500+is_vul*250
        elseif level == 7
            slam_bonus=1000+is_vul*500
        end
        score= game_points+slam_bonus+part_score_bonus+contract_points
        return score
    else
        penalty=0
        if is_vul
            penalty=100*over_trick
        else
            penalty=50*over_trick
        end
        return penalty
    end
end

get_score (generic function with 2 methods)

In [2]:
function raw_score_to_imp(cost)
    cost=abs.(cost)
    if cost<=20
        return 0
    elseif cost<=40
        return 1
    elseif cost<=80
        return 2
    elseif cost<=120
        return 3
    elseif cost<=160
        return 4
    elseif cost<=210
        return 5
    elseif cost<=260
        return 6
    elseif cost<=310
        return 7
    elseif cost<=360
        return 8
    elseif cost<=420
        return 9
    elseif cost<=490
        return 10
    elseif cost<=590
        return 11
    elseif cost<=740
        return 12
    elseif cost<=890
        return 13
    elseif cost<=1090
        return 14
    elseif cost<=1290
        return 15
    elseif cost<=1490
        return 16
    elseif cost<=1740
        return 17
    elseif cost<=1990
        return 18
    elseif cost<=2240
        return 19
    elseif cost<=2490
        return 20
    elseif cost<=2990
        return 21
    elseif cost<=3490
        return 22
    elseif cost<=3990
        return 23
    else
        return 24
    end
end

raw_score_to_imp (generic function with 1 method)

In [3]:
cards_to_indices=Dict{String,Integer}("2"=>1,"3"=>2,"4"=>3,"5"=>4,"6"=>5,"7"=>6,"8"=>7,"9"=>8,"T"=>9,"J"=>10,"Q"=>11,"K"=>12,"A"=>13);
function PBN_to_vector(hand) # PBN formatted a hand is converted to 52-size vector
    raw_data=zeros(52);
    suits=split(hand,'.');
    for i in 1:4
        for card in suits[i]
            raw_data[(i-1)*13+cards_to_indices[string(card)]]=1;
        end
    end
    return raw_data
end

suit_order=Dict{Char,Integer}('N'=> 0,'S'=> 1,'H'=> 2,'D'=> 3,'C'=> 4);
function PBN_to_cost(analysis_list) # PBN formatted a DDS-record is converted to (36,2)-size cost vector
    costs=zeros(36,2);
    for dds in analysis_list
        for i in 2:36
            contract=bids[i];
            trump=contract[2];
            trick_bid= parse(Int32,contract[1])+6;
            # if north is the declarer then east is the leader
            trick_count_n=parse(Int32,dds[suit_order[trump]*4+2],base=16);
            # if south is the declarer then west is the leader
            trick_count_s=parse(Int32,dds[suit_order[trump]*4+4],base=16);
            costs[i,1] +=get_score(contract,trick_count_n);
            costs[i,2] +=get_score(contract,trick_count_s);
        end
    end
    costs=costs/length(analysis_list)
    costs[:,1]= raw_score_to_imp.(costs[:,1] .- maximum(costs[:,1]))./25;
    costs[:,2]= raw_score_to_imp.(costs[:,2] .- maximum(costs[:,2]))./25;
    
    return costs
end

PBN_to_cost (generic function with 1 method)